In [1]:
!pip install ../input/pip-wheels-feedback/transformers-4.20.1-py3-none-any.whl

Processing /kaggle/input/pip-wheels-feedback/transformers-4.20.1-py3-none-any.whl
  Attempting uninstall: transformers
    Found existing installation: transformers 4.18.0
    Uninstalling transformers-4.18.0:
      Successfully uninstalled transformers-4.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.9.3 requires transformers<4.19,>=4.1, but you have transformers 4.20.1 which is incompatible.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModel
import os
import json
import shutil
from types import SimpleNamespace  
import yaml
import multiprocessing as mp
from tqdm.auto import tqdm
import os
import gc

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
N_CORES = mp.cpu_count()

ID_SAMPLE = 0.01

NUM_MODELS = 3

LABEL_MEANS = np.array([0.57056984, 0.25366517, 0.17576499])

LABEL_MEANS[0] *= 1.025
LABEL_MEANS[1] *= 0.875
LABEL_MEANS = LABEL_MEANS / LABEL_MEANS.sum()

sample_submission = pd.read_csv("../input/feedback-prize-effectiveness/sample_submission.csv")


if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    data_folder = "test"
    df = pd.read_csv("../input/feedback-prize-effectiveness/test.csv")
    CALC_SCORE = False
else:
    data_folder = "train"
    df = pd.read_csv("../input/feedback-prize-effectiveness/train.csv")
    df.loc[df.discourse_id == "56744a66949a", "discourse_text"] = "This whole thing is point less how they have us in here for two days im missing my education. We could have finished this in one day and had the rest of the week to get back on the track of learning. I've missed both days of weight lifting, algebra, and my world history that i do not want to fail again! If their are any people actually gonna sit down and take the time to read this then\n\nDO NOT DO THIS NEXT YEAR\n\n.\n\nThey are giving us cold lunches. ham and cheese and an apple, I am 16 years old and my body needs proper food. I wouldnt be complaining if they served actual breakfast. but because of Michelle Obama and her healthy diet rule they surve us 1 poptart in the moring. How does the school board expect us to last from 7:05-12:15 on a pop tart? then expect us to get A's, we are more focused on lunch than anything else. I am about done so if you have the time to read this even though this does not count. Bring PROPER_NAME a big Mac from mc donalds, SCHOOL_NAME, (idk area code but its in LOCATION_NAME)       \xa0    "

    ids = df.essay_id.unique()
    np.random.seed(1337)
    val_ids = np.random.choice(ids, size=int(ID_SAMPLE*len(ids)), replace=False)
    df = df[df.essay_id.isin(val_ids)]
    df = df.reset_index(drop=True)
    CALC_SCORE = True
    
print(CALC_SCORE)
    
df["discourse_type_essay"] = df.groupby("essay_id")["discourse_type"].transform(lambda x: " ".join(x)).values

True


In [4]:
from glob import glob
from tqdm import tqdm
import pandas as pd

essay_texts = {}
for fname in tqdm(glob(f"../input/feedback-prize-effectiveness/{data_folder}/*.txt")):
    with open(fname) as f:
        lines = f.read()
        
    essay_texts[fname.split("/")[-1][:-4]] = lines

df["essay_text"] = df.essay_id.map(essay_texts)

del essay_texts
gc.collect()

df["count"] = df["essay_text"].apply(lambda x: len(x))
df["orig_order"] = range(len(df))
df = df.sort_values(["count", "essay_id", "orig_order"], ascending=True).reset_index(drop=True)

100%|██████████| 4191/4191 [00:21<00:00, 198.15it/s]


In [5]:
initial_df = df.copy()

In [6]:
from torch.utils.data import Dataset, DataLoader
import torch
import collections

class FeedbackDataset(Dataset):

    def __init__(self, df, mode, cfg):
        self.df = df.reset_index(drop=True)
        self.mode = mode
        self.cfg = cfg
        
        self.tokenizer = AutoTokenizer.from_pretrained(cfg.architecture.cache_dir)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        if self.tokenizer.sep_token is None:
            self.tokenizer.sep_token = " "
            
        if hasattr(cfg.dataset, "separator") and len(cfg.dataset.separator):
            self.cfg._tokenizer_sep_token = cfg.dataset.separator
        else:
            self.cfg._tokenizer_sep_token = self.tokenizer.sep_token
                                                       
        self.text = self.get_texts(self.df, self.cfg, self.tokenizer.sep_token)
        
        if self.cfg.tokenizer.lowercase:
            self.df["essay_text"] = self.df["essay_text"].str.lower()
            self.df["discourse_text"] = self.df["discourse_text"].str.lower()

        if self.cfg.dataset.group_discourse:
            grps = self.df.groupby("essay_id", sort=False)
            self.grp_texts = []
            
            s = 0

            for grp in grps.groups:
                g = grps.get_group(grp)
                t = g.essay_text.values[0]
                
                end = 0
                for j in range(len(g)):

                    d = g.discourse_text.values[j]
                    start = t[end:].find(d.strip()) 
                    if start == -1:
                        print("ERROR")
                    
                    start = start + end
                    end = start + len(d.strip())
                    if self.cfg.architecture.aux_type:
                        t = t[:start] + f" [START] " + t[start:end] + " [END] " + t[end:] 
                    elif self.cfg.architecture.use_type:
                        t = t[:start] + f" [START_{g.discourse_type.values[j]}]  " + t[start:end] + f" [END_{g.discourse_type.values[j]}] " + t[end:] 
                    else:
                        t = t[:start] + f" [START] {g.discourse_type.values[j]} " + t[start:end] + " [END] " + t[end:] 

                if hasattr(self.cfg.dataset, "add_group_types") and self.cfg.dataset.add_group_types:
                    t = " ".join(g.discourse_type.values) + f" {self.cfg._tokenizer_sep_token} " + t
                        
                self.grp_texts.append(t)

                s += len(g)

            if self.cfg.dataset.group_discourse:
                
                self.cfg._tokenizer_start_token_id = []
                self.cfg._tokenizer_end_token_id = []

                if self.cfg.architecture.use_type:
                    for type in sorted(self.df.discourse_type.unique()):
                        self.tokenizer.add_tokens([f"[START_{type}]"], special_tokens=True)
                        self.cfg._tokenizer_start_token_id.append(self.tokenizer.encode(f"[START_{type}]")[1])
                    
                    for type in sorted(self.df.discourse_type.unique()):
                        self.tokenizer.add_tokens([f"[END_{type}]"], special_tokens=True)
                        self.cfg._tokenizer_end_token_id.append(self.tokenizer.encode(f"[END_{type}]")[1])

                else:
                    self.tokenizer.add_tokens(["[START]", "[END]"], special_tokens=True)
                    self.cfg._tokenizer_start_token_id.append(self.tokenizer.encode(f"[START]")[1])
                    self.cfg._tokenizer_end_token_id.append(self.tokenizer.encode(f"[END]]")[1])

                print(self.cfg._tokenizer_start_token_id)
                print(self.cfg._tokenizer_end_token_id)

            if hasattr(self.cfg.tokenizer, "add_newline_token") and self.cfg.tokenizer.add_newline_token:
                self.tokenizer.add_tokens([f"\n"], special_tokens=True)

            self.cfg._tokenizer_size = len(self.tokenizer)
            
    def __len__(self):
        if self.cfg.dataset.group_discourse:
            return len(self.grp_texts)
        else:
            return len(self.df)
        
    @staticmethod
    def collate_fn(batch):
        elem = batch[0]

        ret = {}
        for key in elem:
            if key in {"target", "weight"}:
                ret[key] = [d[key].float() for d in batch]
            elif key in {"target_aux"}:

                ret[key] = [d[key].float() for d in batch]
            else:
                ret[key] = torch.stack([d[key] for d in batch], 0)
        return ret
            
    def batch_to_device(batch, device):

        if isinstance(batch, torch.Tensor):
            return batch.to(device)
        elif isinstance(batch, collections.abc.Mapping):
            return {
                key: FeedbackDataset.batch_to_device(value, device)
                for key, value in batch.items()
            }
        elif isinstance(batch, collections.abc.Sequence):
            return [FeedbackDataset.batch_to_device(value, device) for value in batch]
        else:
            raise ValueError(f"Can not move {type(batch)} to device.")
    
    @staticmethod
    def _lowercase(sample):
        if isinstance(sample, str):
            return sample.lower()
        elif isinstance(sample, Iterable):
            return [x.lower() for x in sample]
    
    def get_texts(cls, df, cfg, separator):
        if separator is None:
            if hasattr(cfg.dataset, "separator") and len(cfg.dataset.separator):
                separator = cfg.dataset.separator
            else:
                separator = getattr(cfg, "_tokenizer_sep_token", "<SEPARATOR>")

        lowercase = hasattr(cfg, "tokenizer") and cfg.tokenizer.lowercase
        if isinstance(cfg.dataset.text_column, str):
            texts = df[cfg.dataset.text_column].astype(str)
            if lowercase:
                texts = texts.apply(cls._lowercase)
            texts = texts.values
        else:
            columns = list(cfg.dataset.text_column)
            join_str = f" {separator} "
            texts = df[columns].astype(str)
            if lowercase:
                texts = texts.apply(cls._lowercase)
            texts = texts.apply(lambda x: join_str.join(x), axis=1).values

        return texts
        
    def _read_data(self, idx, sample):

        if self.cfg.dataset.group_discourse:
            text = self.grp_texts[idx]
        else:
            text = self.text[idx]

        if idx == 0:
            print(text)
            
        sample.update(self.encode(text))
        return sample
    
    def encode(self, text):
        sample = dict()
        encodings = self.tokenizer(
            text,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=self.cfg.tokenizer.max_length,
        )
        sample["input_ids"] = encodings["input_ids"][0]
        sample["attention_mask"] = encodings["attention_mask"][0]
        return sample

    def __getitem__(self, idx):
        sample = dict()
            
        sample = self._read_data(idx=idx, sample=sample)

        return sample

In [7]:
from torch import nn
from torch.nn.parameter import Parameter
class NLPAllclsTokenPooling(nn.Module):

    def __init__(self, dim, cfg):
        super(NLPAllclsTokenPooling, self).__init__()

        self.dim = dim
        self.feat_mult = 1
        if cfg.dataset.group_discourse:
            self.feat_mult = 3

    def forward(self, x, attention_mask, input_ids, cfg):

        if not cfg.dataset.group_discourse:
            input_ids_expanded = input_ids.clone().unsqueeze(-1).expand(x.shape)
            attention_mask_expanded = torch.zeros_like(input_ids_expanded)

            attention_mask_expanded[(input_ids_expanded == cfg._tokenizer_cls_token_id) | (input_ids_expanded == cfg._tokenizer_sep_token_id)] = 1

            sum_features = (x * attention_mask_expanded).sum(self.dim)
            ret = sum_features / attention_mask_expanded.sum(self.dim).clip(min=1e-8)

        else:
            ret = []

            for j in range(x.shape[0]):


                idx0 = torch.where((input_ids[j] >= min(cfg._tokenizer_start_token_id)) & (input_ids[j] <= max(cfg._tokenizer_start_token_id)))[0]
                idx1 = torch.where((input_ids[j] >= min(cfg._tokenizer_end_token_id)) & (input_ids[j] <= max(cfg._tokenizer_end_token_id)))[0]

                xx = []
                for jj in range(len(idx0)):
                    xx0 = x[j, idx0[jj]]
                    xx1 = x[j, idx1[jj]]
                    xx2 = x[j, idx0[jj]+1:idx1[jj]].mean(dim=0)
                    xxx = torch.cat([xx0, xx1, xx2]).unsqueeze(0)
                    xx.append(xxx)
                xx = torch.cat(xx)
                ret.append(xx)
        
        return ret

class GeMText(nn.Module):
    def __init__(self, dim, cfg, p=3, eps=1e-6):
        super(GeMText, self).__init__()
        self.dim = dim
        self.p = Parameter(torch.ones(1) * p)
        self.eps = eps
        self.feat_mult = 1

    def forward(self, x, attention_mask, input_ids, cfg):
        attention_mask_expanded = attention_mask.unsqueeze(-1).expand(x.shape)
        x = (x.clamp(min=self.eps) * attention_mask_expanded).pow(self.p).sum(self.dim)
        ret = x / attention_mask_expanded.sum(self.dim).clip(min=self.eps)
        ret = ret.pow(1 / self.p)
        return ret
    
class NLPPoolings:
    _poolings = {
        "All [CLS] token": NLPAllclsTokenPooling,
        "GeM": GeMText
    }
    @classmethod
    def get(cls, name):
        return cls._poolings.get(name)

class FeedbackModel(nn.Module):


    def __init__(self, cfg):

        super(FeedbackModel, self).__init__()

        self.cfg = cfg
        self.n_classes = 3
        config = AutoConfig.from_pretrained(cfg.architecture.cache_dir)
        self.backbone = AutoModel.from_config(config)
    
        self.backbone.pooler = None
        
        if self.cfg.dataset.group_discourse:
            self.backbone.resize_token_embeddings(cfg._tokenizer_size)
        
        self.pooling = NLPPoolings.get(self.cfg.architecture.pool)
        self.pooling = self.pooling(dim=1, cfg=cfg)  # init pooling and pool over token dimension
        
        self.head = nn.Linear(self.backbone.config.hidden_size*self.pooling.feat_mult, self.n_classes)

    def get_features(self, batch):
        attention_mask = batch["attention_mask"]
        input_ids = batch["input_ids"]

        x = self.backbone(
            input_ids=input_ids, attention_mask=attention_mask
        ).last_hidden_state

        x = self.pooling(x, attention_mask, input_ids, cfg=self.cfg)

        if self.cfg.dataset.group_discourse:
            x = torch.cat(x)
        
        if self.cfg.architecture.dropout > 0.0:
            x = F.dropout(x, p=self.cfg.architecture.dropout, training=self.training)

        return x

    def forward(self, batch, calculate_loss=False):
        
        idx = int(torch.where(batch["attention_mask"] == 1)[1].max())
        idx += 1
        batch["attention_mask"] = batch["attention_mask"][:, :idx]
        batch["input_ids"] = batch["input_ids"][:, :idx]
        
        x = self.get_features(batch)
                
        logits = self.head(x)
        outputs = {}

        outputs["logits"] = logits

        return outputs

In [8]:
from transformers import AutoConfig, AutoModel, PreTrainedModel

def create_nlp_backbone(
    cfg, model_class=AutoModel, remove_pooling_layer=False
):

    config = AutoConfig.from_pretrained(
        cfg["backbone"], cache_dir=cfg["cache_dir"]
    )

    kwargs = dict(add_pooling_layer=False) if remove_pooling_layer else dict()
    
    try:
        backbone = model_class.from_config(config, **kwargs)
    except TypeError:
        backbone = model_class.from_config(config)

    return backbone

def glorot_uniform(parameter):
    nn.init.xavier_uniform_(parameter.data, gain=1.0)


class NBMEHead(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NBMEHead, self).__init__()
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.3)
        self.dropout4 = nn.Dropout(0.4)
        self.dropout5 = nn.Dropout(0.5)
        self.classifier = nn.Linear(input_dim, output_dim)
        glorot_uniform(self.classifier.weight)

    def forward(self, x):
        # x is B x S x C
        logits1 = self.classifier(self.dropout1(x))
        logits2 = self.classifier(self.dropout2(x))
        logits3 = self.classifier(self.dropout3(x))
        logits4 = self.classifier(self.dropout4(x))
        logits5 = self.classifier(self.dropout5(x))

        logits = ((logits1 + logits2 + logits3 + logits4 + logits5) / 5)

        return logits

class ModelYauhen(nn.Module):
    def __init__(self, cfg):
        super(ModelYauhen, self).__init__()

        self.cfg = cfg
        self.n_classes = 3
        self.backbone = create_nlp_backbone(
            self.cfg,
            model_class=AutoModel,
            remove_pooling_layer=False,
        )
        self.head = nn.Linear(self.backbone.config.hidden_size, 3)
        if self.cfg["add_wide_dropout"]:
            self.token_type_head = NBMEHead(self.backbone.config.hidden_size, 3)

    def forward(self, batch, calculate_loss=True):
        outputs = {}
        
        idx = int(torch.where(batch["attention_mask"] == 1)[1].max()) 
        idx += 1
        batch["attention_mask"] = batch["attention_mask"][:, :idx]
        batch["input_ids"] = batch["input_ids"][:, :idx]
        batch["word_start_mask"] = batch["word_start_mask"][:, :idx]
        batch["word_ids"] = batch["word_ids"][:, :idx]

        outputs["word_start_mask"] = batch["word_start_mask"]

        x = self.backbone(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        ).last_hidden_state

        for obs_id in range(x.size()[0]):
            for w_id in range(int(torch.max(batch["word_ids"][obs_id]).item()) + 1):
                chunk_mask = batch["word_ids"][obs_id] == w_id
                chunk_logits = x[obs_id] * chunk_mask.unsqueeze(-1)
                chunk_logits = chunk_logits.sum(dim=0) / chunk_mask.sum()
                x[obs_id][chunk_mask] = chunk_logits

        if self.cfg["add_wide_dropout"]:
            logits = self.token_type_head(x)
        else:
            logits = self.head(x)
        outputs["logits"] = logits      

        return outputs
    
    
from torch.utils.data import Dataset
from transformers import AutoTokenizer


class FeedbackDatasetYauhen(Dataset):
    @staticmethod
    def _lowercase(sample):
        if isinstance(sample, str):
            return sample.lower()
        elif isinstance(sample, Iterable):
            return [x.lower() for x in sample]

    def __init__(self, df, cfg):
        self.cfg = cfg
        self.df = df

        self.tokenizer = AutoTokenizer.from_pretrained(
            self.cfg["backbone"],
            add_prefix_space=True,
            use_fast=True,
            cache_dir=self.cfg["cache_dir"],
        )

        self.text = self.get_texts(self.df, self.cfg, self.tokenizer.sep_token)
        self.labels = self.df["tokens"].values

    @classmethod
    def get_texts(cls, df, cfg, separator=None):
        texts = df[cfg["text_column"]].values

        if cfg["lowercase"]:
            texts = [cls._lowercase(x) for x in texts]

        return texts

    def __getitem__(self, idx):
        sample = dict()
            
        text = self.text[idx]
        
        if "deberta-v3" in self.cfg["backbone"]:
            text = [x.replace("\n", "[NL_HYDRO]") for x in list(text)]
            text = [x if not x.isspace() else "[SP_HYDRO]" * len(x) for x in text]
            tokenizer_input = [text]
            raise ValueError(f"BES {text}")
        else:
            if "add_types" in self.cfg and self.cfg["add_types"]:
                tokenizer_input = [x if x_idx > 0 else x + self.tokenizer.sep_token for x_idx, x in enumerate(list(text))]
            else:
                tokenizer_input = [list(text)]

        encodings = self.tokenizer(
            tokenizer_input,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=2048,
            is_split_into_words=True,
        )

        sample["input_ids"] = encodings["input_ids"][0]
        sample["attention_mask"] = encodings["attention_mask"][0]

        word_ids = encodings.word_ids(0)
        word_ids = [-1 if x is None else x for x in word_ids]
        sample["word_ids"] = torch.tensor(word_ids)

        word_start_mask = []
        lab_idx = -1
        for i, word in enumerate(word_ids):
            word_start = word > -1 and (i == 0 or word_ids[i - 1] != word)
            if word_start:
                lab_idx += 1
                if self.labels[idx][lab_idx] != 1:
                    word_start_mask.append(True)
                    continue

            word_start_mask.append(False)

        sample["word_start_mask"] = torch.tensor(word_start_mask)

        return sample
    
    def __len__(self):
        return len(self.df)

In [9]:
test = df.copy()

In [10]:
from tqdm import tqdm

all_obs = []

for name, gr in tqdm(test.groupby("essay_id", sort=False)):
    essay_text_start_end = gr.essay_text.values[0]
    token_labels = []
    token_obs = []
    end_pos = 0
    
    for idx, row in gr.reset_index(drop=True).iterrows():
        target_text = row["discourse_type"] + " " + row["discourse_text"].strip()
        
        essay_text_start_end = essay_text_start_end[:end_pos] + essay_text_start_end[end_pos:].replace(row["discourse_text"].strip(), target_text, 1)
        
        start_pos = essay_text_start_end[end_pos:].find(target_text)
        if start_pos == -1:
            raise ValueError()
        start_pos += end_pos
        
        if idx == 0 and start_pos > 0:
            token_labels.append(1)
            token_obs.append(essay_text_start_end[:start_pos])
        
        if start_pos > end_pos and end_pos > 0:
            token_labels.append(1)
            token_obs.append(essay_text_start_end[end_pos:start_pos])
  
        end_pos = start_pos + len(target_text)
        token_labels.append(0)
        token_obs.append(essay_text_start_end[start_pos: end_pos])
            
        if idx == len(gr) - 1 and end_pos < len(essay_text_start_end):
            token_labels.append(1)
            token_obs.append(essay_text_start_end[end_pos:])
            
    if len(token_labels) != len(token_obs):
        raise ValueError()
            
    all_obs.append((name, token_labels, token_obs))

tt = pd.DataFrame(all_obs, columns=["essay_id", "tokens", "essay_text"])

100%|██████████| 41/41 [00:00<00:00, 1034.65it/s]


In [11]:
from tqdm import tqdm

all_obs = []

for name, gr in tqdm(test.groupby("essay_id", sort=False)):
    essay_text_start_end = gr.essay_text.values[0]
    token_labels = []
    token_obs = []
    end_pos = 0
    
    token_obs.append(" ".join(gr.discourse_type.to_list()))
    token_labels.append(1)
    
    for idx, row in gr.reset_index(drop=True).iterrows():
        target_text = row["discourse_type"] + " " + row["discourse_text"].strip()
        
        essay_text_start_end = essay_text_start_end[:end_pos] + essay_text_start_end[end_pos:].replace(row["discourse_text"].strip(), target_text, 1)
        
        start_pos = essay_text_start_end[end_pos:].find(target_text)
        if start_pos == -1:
            raise ValueError()
        start_pos += end_pos
        
        if idx == 0 and start_pos > 0:
            token_labels.append(1)
            token_obs.append(essay_text_start_end[:start_pos])
        
        if start_pos > end_pos and end_pos > 0:
            token_labels.append(1)
            token_obs.append(essay_text_start_end[end_pos:start_pos])
  
        end_pos = start_pos + len(target_text)
        token_labels.append(0)
        token_obs.append(essay_text_start_end[start_pos: end_pos])
            
        if idx == len(gr) - 1 and end_pos < len(essay_text_start_end):
            token_labels.append(1)
            token_obs.append(essay_text_start_end[end_pos:])
            
    if len(token_labels) != len(token_obs):
        raise ValueError()
            
    all_obs.append((name, token_labels, token_obs))

tt_v2 = pd.DataFrame(all_obs, columns=["essay_id", "tokens", "essay_text"])

100%|██████████| 41/41 [00:00<00:00, 1002.97it/s]


In [12]:
def run_predictions_philipp(exp_name, df, BS=1, num_models=NUM_MODELS):
    
    cfg = yaml.safe_load(open(f"../input/{exp_name}/cfg.yaml").read())
    for k,v in cfg.items():
        if type(v) == dict:
            cfg[k] = SimpleNamespace(**v)
    cfg = SimpleNamespace(**cfg)

    if cfg.architecture.backbone == 'microsoft/deberta-v3-large':
        cfg.architecture.cache_dir = "../input/deberta-v3-large/"
    elif cfg.architecture.backbone == 'microsoft/deberta-v3-small':
        cfg.architecture.cache_dir = "../input/deberta-v3-small/"
    elif cfg.architecture.backbone == 'microsoft/deberta-v3-base':
        cfg.architecture.cache_dir = "../input/deberta-v3-lbase/"

    ds = FeedbackDataset(df.iloc[:], mode="test", cfg=cfg)
    
    preds_all = []
    for fold in range(num_models):
        print(f"running model {fold}")
        
        model = FeedbackModel(cfg).to("cuda").eval()
    
        d = torch.load(f"../input/{exp_name}/checkpoint-fold{fold}.pth", map_location="cpu")

        model_weights = d["model"]
        model_weights = {k.replace("module.", ""): v for k, v in model_weights.items()}
        
        for k in list(model_weights.keys()):
            if "aux" in k or "loss_fn" in k:
                del model_weights[k]

        model.load_state_dict(collections.OrderedDict(model_weights), strict=True)
        
        del d
        del model_weights
        gc.collect() 
    
        batch_size = BS
        dl = DataLoader(ds, shuffle=False, batch_size = batch_size, num_workers = N_CORES)

        with torch.no_grad():    
            preds = []
            for batch in tqdm(dl):

                batch = FeedbackDataset.batch_to_device(batch, "cuda")
                out = model(batch)
                preds.append(out["logits"].float().softmax(dim=1).detach().cpu().numpy())

        preds_all.append(np.concatenate(preds, axis=0))
        
        del model
        del dl
        gc.collect()
        
    del ds
    
    
    preds = np.mean(preds_all, axis=0)
    
    return preds

def run_predictions_yauhen(all_cfgs, df, yauhen_batch_size=1):
    ds = FeedbackDatasetYauhen(df=df, cfg=all_cfgs[0])
        
    preds_all = []

    for params in all_cfgs:

        model = ModelYauhen(params).to("cuda").eval()

        d = torch.load(params["path"], map_location="cpu")

        model_weights = d["model"]
        model_weights = {k.replace("module.", ""): v for k, v in model_weights.items()}
        model.load_state_dict(collections.OrderedDict(model_weights), strict=True)
        
        del d
        del model_weights
        gc.collect() 
    
        dl = DataLoader(ds, shuffle=False, batch_size = yauhen_batch_size, num_workers = N_CORES)

        with torch.no_grad():
            
            preds = []
            for batch in tqdm(dl):
                texts = {
                key: value.to("cuda")
                for key, value in batch.items()
            }
                output = model.forward(texts, calculate_loss=False)

                val = (
                        torch.softmax(output["logits"][output["word_start_mask"]], dim=1).detach().cpu().numpy()
                    )

                preds.append(val)

        preds_all.append(np.concatenate(preds, axis=0))
        
        del model
        del dl
        gc.collect()
        
    del ds
    
    preds = np.mean(preds_all, axis=0)
    
    return preds

In [13]:
preds = []
weights = []

In [14]:
def scale_probs(pp_single):
    pp = pp_single.copy()

    for _ in range(100):

        pp = pp * (LABEL_MEANS.reshape(1,3) / pp.mean(axis=0))

        pp = pp / pp.sum(axis=1, keepdims=True)
        
    return pp

In [15]:
import lightgbm
def get_features(df, vects):
    vect_discourse = vects["vect_discourse"]
    X = vect_discourse.transform(df["discourse_text"]).A

    X = np.concatenate([X, X.sum(axis=1).reshape(-1,1)], axis=1)

    vect_essay = vects["vect_essay"]
    XX = vect_essay.transform(df["essay_text"]).A
    XX = np.concatenate([XX, XX.sum(axis=1).reshape(-1,1)], axis=1)
    X = np.concatenate([X, XX], axis=1)

    vect_type = vects["vect_type"]
    X = np.concatenate([X, vect_type.transform(df["discourse_type"]).A], axis=1)

    vect_type_essay = vects["vect_type_essay"]
    X = np.concatenate([X, vect_type_essay.transform(df["discourse_type_essay"]).A], axis=1)

    f = "rel_rank"
    X = np.concatenate([X, df[f].values.reshape(-1,1)], axis=1)
    print(X.shape)

    return X

def run_predictions_lgb(exp_name, df):
    df["rank"] = df.groupby("essay_id")["discourse_type"].transform(lambda x: np.arange(len(x))).values
    df["length"] = df.groupby("essay_id")["discourse_type"].transform(lambda x: len(x)).values
    df["rel_rank"] = df["rank"] / df["length"]
    
    pps = []
    for fold in [-1]:
    
        vects = pd.read_pickle(f"../input/{exp_name}/fold{fold}/vectorizers.p")

        X = get_features(df, vects)

        clf = lightgbm.Booster(model_file=f"../input/{exp_name}/fold{fold}/model_seed0.txt")

        preds = clf.predict(X)
        pps.append(preds)
    preds = np.mean(pps, axis=0)
    
    print(preds.mean(axis=0))
    
    return preds

preds.append(scale_probs(run_predictions_lgb("lgb-v0", df)))
weights.append(1.131471)

(358, 6612)
[0.56489283 0.27617336 0.15893382]


In [16]:
cfg_1 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/pseudo-75-datasets-v4/olivine-spaniel-ff/checkpoint-fold0.pth",
         "add_wide_dropout": False,
      }

cfg_2 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/pseudo-75-datasets-v4/olivine-spaniel-ff/checkpoint-fold1.pth",
         "add_wide_dropout": False,
      }

cfg_3 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/pseudo-75-datasets-v4/olivine-spaniel-ff/checkpoint-fold2.pth",
         "add_wide_dropout": False,
      }

cfg_4 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/pseudo-75-datasets-v4/olivine-spaniel-ff/checkpoint-fold3.pth",
         "add_wide_dropout": False,
      }

cfg_5 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/pseudo-75-datasets-v4/olivine-spaniel-ff/checkpoint-fold4.pth",
         "add_wide_dropout": False,
      }

all_cfgs = [cfg_1, cfg_2, cfg_3, cfg_4, cfg_5]

preds.append(scale_probs(run_predictions_yauhen(all_cfgs, tt, yauhen_batch_size=4)))
weights.append(-1.795161)

100%|██████████| 11/11 [00:05<00:00,  2.19it/s]


In [17]:
cfg_1 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/meteoric-bettong-v2-ff/checkpoint-fold0.pth",
         "add_wide_dropout": True,
      }

cfg_2 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/meteoric-bettong-v2-ff/checkpoint-fold1.pth",
         "add_wide_dropout": True,
      }

cfg_3 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/meteoric-bettong-v2-ff/checkpoint-fold2.pth",
         "add_wide_dropout": True,
      }

all_cfgs = [cfg_1, cfg_2, cfg_3]

preds.append(scale_probs(run_predictions_yauhen(all_cfgs, tt)))
weights.append(-0.455578)

100%|██████████| 41/41 [00:05<00:00,  7.50it/s]


In [18]:
cfg_1 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/saffron-rook-ff/checkpoint_fold_0.pth",
         "add_wide_dropout": True,
         "add_types": True,
      }

cfg_2 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/saffron-rook-ff/checkpoint_fold_1.pth",
         "add_wide_dropout": True,
         "add_types": True,
      }

cfg_3 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/saffron-rook-ff/checkpoint_fold_2.pth",
         "add_wide_dropout": True,
         "add_types": True,
      }

all_cfgs = [cfg_1, cfg_2, cfg_3]

seed_1 = run_predictions_yauhen(all_cfgs, tt_v2, yauhen_batch_size=4)

cfg_1 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/saffron-rook-v2-ff/checkpoint_fold_0.pth",
         "add_wide_dropout": True,
         "add_types": True,
      }

cfg_2 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/saffron-rook-v2-ff/checkpoint_fold_1.pth",
         "add_wide_dropout": True,
         "add_types": True,
      }

cfg_3 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/saffron-rook-v2-ff/checkpoint_fold_2.pth",
         "add_wide_dropout": True,
         "add_types": True,
      }

all_cfgs = [cfg_1, cfg_2, cfg_3]

seed_2 = run_predictions_yauhen(all_cfgs, tt_v2, yauhen_batch_size=4)

seeds = (seed_1 + seed_2) / 2

preds.append(scale_probs(seeds))
weights.append(0.639652)

100%|██████████| 11/11 [00:05<00:00,  2.03it/s]


In [19]:
cfg_1 = {"backbone": "../input/debertaxlarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertaxlarge",
       "path": "../input/big-ocelot-ff/checkpoint_fold_0.pth",
         "add_wide_dropout": False,
         "add_types": True,
      }

cfg_2 = {"backbone": "../input/debertaxlarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertaxlarge",
       "path": "../input/big-ocelot-ff/checkpoint_fold_1.pth",
         "add_wide_dropout": False,
         "add_types": True,
      }

cfg_3 = {"backbone": "../input/debertaxlarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertaxlarge",
       "path": "../input/big-ocelot-ff/checkpoint_fold_2.pth",
         "add_wide_dropout": False,
         "add_types": True,
      }

cfg_4 = {"backbone": "../input/debertaxlarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertaxlarge",
       "path": "../input/big-ocelot-ff-v3/checkpoint-fold0.pth",
         "add_wide_dropout": False,
         "add_types": True,
      }

cfg_5 = {"backbone": "../input/debertaxlarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertaxlarge",
       "path": "../input/big-ocelot-ff-v3/checkpoint-fold1.pth",
         "add_wide_dropout": False,
         "add_types": True,
      }

all_cfgs = [cfg_1, cfg_2, cfg_3, cfg_4, cfg_5]

preds.append(scale_probs(run_predictions_yauhen(all_cfgs, tt_v2, yauhen_batch_size=4)))
weights.append(1.586749)

100%|██████████| 11/11 [00:10<00:00,  1.05it/s]


In [20]:
cfg_1 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/shrewd-rook-3ep-ff/checkpoint-fold0.pth",
         "add_wide_dropout": False,
      }

cfg_2 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/shrewd-rook-3ep-ff/checkpoint-fold1.pth",
         "add_wide_dropout": False,
      }

cfg_3 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/shrewd-rook-3ep-ff/checkpoint-fold2.pth",
         "add_wide_dropout": False,
      }

cfg_4 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/shrewd-rook-3ep-ff-v2/checkpoint-fold0.pth",
         "add_wide_dropout": False,
      }

cfg_5 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/shrewd-rook-3ep-ff-v2/checkpoint-fold1.pth",
         "add_wide_dropout": False,
      }

cfg_6 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/shrewd-rook-3ep-ff-v2/checkpoint-fold2.pth",
         "add_wide_dropout": False,
      }

all_cfgs = [cfg_1, cfg_2, cfg_3, cfg_4, cfg_5, cfg_6]

preds.append(scale_probs(run_predictions_yauhen(all_cfgs, tt, yauhen_batch_size=4)))
weights.append(0.983297)

100%|██████████| 11/11 [00:05<00:00,  2.16it/s]


In [21]:
cfg_1 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/conscious-uakari-ff/checkpoint-fold0.pth",
         "add_wide_dropout": True,
         "add_types": True,
      }

cfg_2 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/conscious-uakari-ff/checkpoint-fold1.pth",
         "add_wide_dropout": True,
         "add_types": True,
      }

cfg_3 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/conscious-uakari-ff/checkpoint-fold2.pth",
         "add_wide_dropout": True,
         "add_types": True,
      }

cfg_4 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/conscious-uakari-ff-v2/checkpoint-fold0.pth",
         "add_wide_dropout": True,
         "add_types": True,
      }

cfg_5 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/conscious-uakari-ff-v2/checkpoint-fold1.pth",
         "add_wide_dropout": True,
         "add_types": True,
      }

cfg_6 = {"backbone": "../input/debertalarge",
       "lowercase": False,
       "text_column": "essay_text",
       "cache_dir": "../input/debertalarge",
       "path": "../input/conscious-uakari-ff-v2/checkpoint-fold2.pth",
         "add_wide_dropout": True,
         "add_types": True,
      }

all_cfgs = [cfg_1, cfg_2, cfg_3, cfg_4, cfg_5, cfg_6]


preds.append(scale_probs(run_predictions_yauhen(all_cfgs, tt_v2, yauhen_batch_size=4)))
weights.append(3.707194)

100%|██████████| 11/11 [00:05<00:00,  2.04it/s]


In [22]:
preds.append(scale_probs(run_predictions_philipp("valiant-degu-ff-2", df, BS=8)))
weights.append(0.590190)

/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


running model 0


  0%|          | 0/45 [00:00<?, ?it/s]

Lead [SEP] Lead Position Evidence Claim Claim Evidence Concluding Statement [SEP] Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. 


100%|██████████| 45/45 [00:08<00:00,  5.32it/s]


...

100%|██████████| 45/45 [00:08<00:00,  5.37it/s]


running model 2


  0%|          | 0/45 [00:00<?, ?it/s]

Lead [SEP] Lead Position Evidence Claim Claim Evidence Concluding Statement [SEP] Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. 


100%|██████████| 45/45 [00:08<00:00,  5.26it/s]


In [23]:
preds.append(scale_probs(run_predictions_philipp("axiomatic-vulture-ff-v2", df, BS=8, num_models=5)))
weights.append(0.964377)

[128001]
[128002]
running model 0


  0%|          | 0/6 [00:00<?, ?it/s]

Lead Position Evidence Claim Claim Evidence Concluding Statement [SEP]  [START] Lead Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. [END] 

 [START] Position In the story it states that the faces up there are olny land forms [END] .  [START] Evidence Land forms are shaped over thousinds of years. As the years go on the land form gets smaller and smaller. [END]   [START] Claim Also the planet can not suport life. [END] 

 [START] Claim All the consperisetheris out ther dont know what they are talking about [END] .

 [START] Evidence They dont have the evidens to provide like everone else hase. nasa is a groop of scintest the consperisetheris are a groop of craze people. [END] 

 [START] Concluding Statement thes is why you shud not blesve everthing you see on the internet. lisint to 

100%|██████████| 6/6 [00:05<00:00,  1.04it/s]


running model 1


...

100%|██████████| 6/6 [00:05<00:00,  1.05it/s]


running model 4


  0%|          | 0/6 [00:00<?, ?it/s]

Lead Position Evidence Claim Claim Evidence Concluding Statement [SEP]  [START] Lead Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. [END] 

 [START] Position In the story it states that the faces up there are olny land forms [END] .  [START] Evidence Land forms are shaped over thousinds of years. As the years go on the land form gets smaller and smaller. [END]   [START] Claim Also the planet can not suport life. [END] 

 [START] Claim All the consperisetheris out ther dont know what they are talking about [END] .

 [START] Evidence They dont have the evidens to provide like everone else hase. nasa is a groop of scintest the consperisetheris are a groop of craze people. [END] 

 [START] Concluding Statement thes is why you shud not blesve everthing you see on the internet. lisint to 

100%|██████████| 6/6 [00:05<00:00,  1.03it/s]


In [24]:
preds.append(scale_probs(run_predictions_philipp("smart-bumblebee-ff", df)))
weights.append(0.366988)

[128001]
[128002]
running model 0


  0%|          | 0/41 [00:00<?, ?it/s]

Lead Position Evidence Claim Claim Evidence Concluding Statement [SEP]  [START] Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. [END] 

 [START] In the story it states that the faces up there are olny land forms [END] .  [START] Land forms are shaped over thousinds of years. As the years go on the land form gets smaller and smaller. [END]   [START] Also the planet can not suport life. [END] 

 [START] All the consperisetheris out ther dont know what they are talking about [END] .

 [START] They dont have the evidens to provide like everone else hase. nasa is a groop of scintest the consperisetheris are a groop of craze people. [END] 

 [START] thes is why you shud not blesve everthing you see on the internet. lisint to the scintest. leav them in charge [END]  .


100%|██████████| 41/41 [00:06<00:00,  6.40it/s]


running model 1


...

100%|██████████| 41/41 [00:06<00:00,  6.35it/s]


running model 2


  0%|          | 0/41 [00:00<?, ?it/s]

Lead Position Evidence Claim Claim Evidence Concluding Statement [SEP]  [START] Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. [END] 

 [START] In the story it states that the faces up there are olny land forms [END] .  [START] Land forms are shaped over thousinds of years. As the years go on the land form gets smaller and smaller. [END]   [START] Also the planet can not suport life. [END] 

 [START] All the consperisetheris out ther dont know what they are talking about [END] .

 [START] They dont have the evidens to provide like everone else hase. nasa is a groop of scintest the consperisetheris are a groop of craze people. [END] 

 [START] thes is why you shud not blesve everthing you see on the internet. lisint to the scintest. leav them in charge [END]  .


100%|██████████| 41/41 [00:06<00:00,  6.31it/s]


In [25]:
preds.append(scale_probs(run_predictions_philipp("awesome-rose-ff-v2", df, BS=8, num_models=5)))
weights.append(1.162001)

[128001]
[128002]
running model 0


  0%|          | 0/6 [00:00<?, ?it/s]

 [START] Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. [END] 

 [START] In the story it states that the faces up there are olny land forms [END] .  [START] Land forms are shaped over thousinds of years. As the years go on the land form gets smaller and smaller. [END]   [START] Also the planet can not suport life. [END] 

 [START] All the consperisetheris out ther dont know what they are talking about [END] .

 [START] They dont have the evidens to provide like everone else hase. nasa is a groop of scintest the consperisetheris are a groop of craze people. [END] 

 [START] thes is why you shud not blesve everthing you see on the internet. lisint to the scintest. leav them in charge [END]  .


100%|██████████| 6/6 [00:05<00:00,  1.11it/s]


running model 1


...

100%|██████████| 6/6 [00:05<00:00,  1.13it/s]


running model 4


  0%|          | 0/6 [00:00<?, ?it/s]

 [START] Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. [END] 

 [START] In the story it states that the faces up there are olny land forms [END] .  [START] Land forms are shaped over thousinds of years. As the years go on the land form gets smaller and smaller. [END]   [START] Also the planet can not suport life. [END] 

 [START] All the consperisetheris out ther dont know what they are talking about [END] .

 [START] They dont have the evidens to provide like everone else hase. nasa is a groop of scintest the consperisetheris are a groop of craze people. [END] 

 [START] thes is why you shud not blesve everthing you see on the internet. lisint to the scintest. leav them in charge [END]  .


100%|██████████| 6/6 [00:05<00:00,  1.12it/s]


In [26]:
preds.append(scale_probs(run_predictions_philipp("honest-apple-ff", df, BS=8)))
weights.append(0.543224)

[128001]
[128002]
running model 0


  0%|          | 0/6 [00:00<?, ?it/s]

Lead Position Evidence Claim Claim Evidence Concluding Statement [SEP]  [START] Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. [END] 

 [START] In the story it states that the faces up there are olny land forms [END] .  [START] Land forms are shaped over thousinds of years. As the years go on the land form gets smaller and smaller. [END]   [START] Also the planet can not suport life. [END] 

 [START] All the consperisetheris out ther dont know what they are talking about [END] .

 [START] They dont have the evidens to provide like everone else hase. nasa is a groop of scintest the consperisetheris are a groop of craze people. [END] 

 [START] thes is why you shud not blesve everthing you see on the internet. lisint to the scintest. leav them in charge [END]  .


100%|██████████| 6/6 [00:05<00:00,  1.07it/s]


running model 1


...

100%|██████████| 6/6 [00:05<00:00,  1.08it/s]


running model 2


  0%|          | 0/6 [00:00<?, ?it/s]

Lead Position Evidence Claim Claim Evidence Concluding Statement [SEP]  [START] Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. [END] 

 [START] In the story it states that the faces up there are olny land forms [END] .  [START] Land forms are shaped over thousinds of years. As the years go on the land form gets smaller and smaller. [END]   [START] Also the planet can not suport life. [END] 

 [START] All the consperisetheris out ther dont know what they are talking about [END] .

 [START] They dont have the evidens to provide like everone else hase. nasa is a groop of scintest the consperisetheris are a groop of craze people. [END] 

 [START] thes is why you shud not blesve everthing you see on the internet. lisint to the scintest. leav them in charge [END]  .


100%|██████████| 6/6 [00:05<00:00,  1.06it/s]


In [27]:
preds.append(scale_probs(run_predictions_philipp("funky-funk-ff", df, BS=8)))
weights.append(1.455657)

[128001]
[128002]
running model 0


  0%|          | 0/6 [00:00<?, ?it/s]

Lead Position Evidence Claim Claim Evidence Concluding Statement [SEP]  [START] Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. [END] 

 [START] In the story it states that the faces up there are olny land forms [END] .  [START] Land forms are shaped over thousinds of years. As the years go on the land form gets smaller and smaller. [END]   [START] Also the planet can not suport life. [END] 

 [START] All the consperisetheris out ther dont know what they are talking about [END] .

 [START] They dont have the evidens to provide like everone else hase. nasa is a groop of scintest the consperisetheris are a groop of craze people. [END] 

 [START] thes is why you shud not blesve everthing you see on the internet. lisint to the scintest. leav them in charge [END]  .


100%|██████████| 6/6 [00:05<00:00,  1.05it/s]


running model 1


...

100%|██████████| 6/6 [00:05<00:00,  1.08it/s]


running model 2


  0%|          | 0/6 [00:00<?, ?it/s]

Lead Position Evidence Claim Claim Evidence Concluding Statement [SEP]  [START] Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. [END] 

 [START] In the story it states that the faces up there are olny land forms [END] .  [START] Land forms are shaped over thousinds of years. As the years go on the land form gets smaller and smaller. [END]   [START] Also the planet can not suport life. [END] 

 [START] All the consperisetheris out ther dont know what they are talking about [END] .

 [START] They dont have the evidens to provide like everone else hase. nasa is a groop of scintest the consperisetheris are a groop of craze people. [END] 

 [START] thes is why you shud not blesve everthing you see on the internet. lisint to the scintest. leav them in charge [END]  .


100%|██████████| 6/6 [00:05<00:00,  1.07it/s]


In [28]:
preds.append(scale_probs(run_predictions_philipp("lame-flame-ff-v2", df, BS=8, num_models=5)))
weights.append(1.981731)

[128001]
[128002]
running model 0


  0%|          | 0/6 [00:00<?, ?it/s]

Lead Position Evidence Claim Claim Evidence Concluding Statement [SEP]  [START] Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. [END] 

 [START] In the story it states that the faces up there are olny land forms [END] .  [START] Land forms are shaped over thousinds of years. As the years go on the land form gets smaller and smaller. [END]   [START] Also the planet can not suport life. [END] 

 [START] All the consperisetheris out ther dont know what they are talking about [END] .

 [START] They dont have the evidens to provide like everone else hase. nasa is a groop of scintest the consperisetheris are a groop of craze people. [END] 

 [START] thes is why you shud not blesve everthing you see on the internet. lisint to the scintest. leav them in charge [END]  .


100%|██████████| 6/6 [00:05<00:00,  1.07it/s]


running model 1


...

100%|██████████| 6/6 [00:05<00:00,  1.08it/s]


running model 4


  0%|          | 0/6 [00:00<?, ?it/s]

Lead Position Evidence Claim Claim Evidence Concluding Statement [SEP]  [START] Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. [END] 

 [START] In the story it states that the faces up there are olny land forms [END] .  [START] Land forms are shaped over thousinds of years. As the years go on the land form gets smaller and smaller. [END]   [START] Also the planet can not suport life. [END] 

 [START] All the consperisetheris out ther dont know what they are talking about [END] .

 [START] They dont have the evidens to provide like everone else hase. nasa is a groop of scintest the consperisetheris are a groop of craze people. [END] 

 [START] thes is why you shud not blesve everthing you see on the internet. lisint to the scintest. leav them in charge [END]  .


100%|██████████| 6/6 [00:05<00:00,  1.08it/s]


In [29]:
preds.append(scale_probs(run_predictions_philipp("pastel-frog-ff", df, BS=8)))
weights.append(-1.005743)

[128001]
[128002]
running model 0


  0%|          | 0/6 [00:00<?, ?it/s]

Lead Position Evidence Claim Claim Evidence Concluding Statement [SEP]  [START] Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. [END] 

 [START] In the story it states that the faces up there are olny land forms [END] .  [START] Land forms are shaped over thousinds of years. As the years go on the land form gets smaller and smaller. [END]   [START] Also the planet can not suport life. [END] 

 [START] All the consperisetheris out ther dont know what they are talking about [END] .

 [START] They dont have the evidens to provide like everone else hase. nasa is a groop of scintest the consperisetheris are a groop of craze people. [END] 

 [START] thes is why you shud not blesve everthing you see on the internet. lisint to the scintest. leav them in charge [END]  .


100%|██████████| 6/6 [00:01<00:00,  3.96it/s]


running model 1


...

100%|██████████| 6/6 [00:01<00:00,  4.02it/s]


running model 2


  0%|          | 0/6 [00:00<?, ?it/s]

Lead Position Evidence Claim Claim Evidence Concluding Statement [SEP]  [START] Why do you think ther is a face on Red Planet called Cydonia. Well ther is not the olny thing that is up ther is a wast land of nothing. Dont let consperisetheres out ther fool you. I will show you the facts and prove to you ther are no aliens up there. [END] 

 [START] In the story it states that the faces up there are olny land forms [END] .  [START] Land forms are shaped over thousinds of years. As the years go on the land form gets smaller and smaller. [END]   [START] Also the planet can not suport life. [END] 

 [START] All the consperisetheris out ther dont know what they are talking about [END] .

 [START] They dont have the evidens to provide like everone else hase. nasa is a groop of scintest the consperisetheris are a groop of craze people. [END] 

 [START] thes is why you shud not blesve everthing you see on the internet. lisint to the scintest. leav them in charge [END]  .


100%|██████████| 6/6 [00:01<00:00,  3.94it/s]


In [30]:
preds_orig = np.array(preds).copy()

In [31]:
df_ind_models = df.copy()

for model_idx in range(len(preds)):
    df_ind_models[f"Adequate_{model_idx}"] = preds[model_idx][:,0]
    df_ind_models[f"Effective_{model_idx}"] = preds[model_idx][:,1]
    df_ind_models[f"Ineffective_{model_idx}"] = preds[model_idx][:,2]

In [32]:
preds = np.average(preds, weights=weights, axis=0)

In [33]:
assert len(preds) == len(df)

In [34]:
pp = preds.copy()

eps = 0.0001
pp = pp.clip(eps, 1 - eps)
pp = pp / pp.sum(axis=1, keepdims=True)

pp = scale_probs(pp)

In [35]:
df["Adequate"] = pp[:, 0] 
df["Effective"] = pp[:, 1] 
df["Ineffective"] = pp[:, 2] 

In [36]:
df_ind_models["Adequate"] = pp[:, 0] 
df_ind_models["Effective"] = pp[:, 1] 
df_ind_models["Ineffective"] = pp[:, 2] 

In [37]:
orig_preds = pp.copy()

In [38]:
if CALC_SCORE:
    from sklearn.metrics import log_loss
    
    label_cols = ["Adequate", "Effective", "Ineffective"]
    
    y = np.zeros_like(preds)
    
    for ii, jj in enumerate([label_cols.index(x) for x in df["discourse_effectiveness"].values]):
        y[ii,jj] = 1
        
    print(log_loss(y, pp))

0.34463156944186635


In [39]:
label_cols = ["Adequate", "Effective", "Ineffective"]
oof_cols = []
for j, l in enumerate(label_cols):

    df[f"oof_{l}"] = pp[:,j]
    oof_cols.append(f"oof_{l}")

In [40]:
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import torch

import torch.nn as nn

class FeedbackStackerModel(nn.Module):
    def __init__(self, n_features):
        super(FeedbackStackerModel, self).__init__()
        
        self.sizes = [256, 128, 64]
        
        self.features = nn.Sequential(
            nn.utils.weight_norm(nn.Linear(n_features, self.sizes[0])),
            nn.PReLU(),
            nn.Linear(self.sizes[0], self.sizes[1]),
            nn.PReLU(),
            nn.Linear(self.sizes[1], self.sizes[2]),
            nn.PReLU(),
        )
        
        self.head = nn.Linear(self.sizes[-1], 3)
        
        self.loss_fn = nn.CrossEntropyLoss()
        
    def forward(self, x, y):
        
        x = self.features(x)
        x = self.head(x)
        
        
        output = {}
        
        output["logits"] = x
        
        if self.training:
            output["loss"] = self.loss_fn(x, y.argmax(dim=1))
        
        return output

class FeedbackStackerDataset(Dataset):

    def __init__(self, df, mode):
        self.df = df.copy().reset_index(drop=True)
        self.mode = mode

        self.feature_cols = oof_cols.copy()
        self.label_cols = label_cols.copy()
        
        df = self.df
        
        df[f"len"] = df.groupby("essay_id")[f"discourse_id"].transform("count") / 10
        self.feature_cols.append(f"len")
        
        for j, l in enumerate(label_cols):
            df[f"oof_{l}_mean"] = df.groupby("essay_id")[f"oof_{l}"].transform("mean")
            self.feature_cols.append(f"oof_{l}_mean")
            
            df[f"oof_{l}_t_mean"] = df.groupby(["essay_id", "discourse_type"])[f"oof_{l}"].transform("mean")
            self.feature_cols.append(f"oof_{l}_t_mean")

        self.num_features = len(self.feature_cols)

        self.X = self.df[self.feature_cols].values
        self.y = self.df[self.label_cols].values

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        X = self.X[idx]
        y = self.y[idx]
        
        return torch.FloatTensor(X), torch.FloatTensor(y)


    def __len__(self):
        return self.df.shape[0]
    
ds = FeedbackStackerDataset(df.copy(), mode="val")
ds[0][0].shape

def run_nn_stacker(exp_name, df, BS=64):


    ds = FeedbackStackerDataset(df.iloc[:].copy(), mode="test")
    
    checkpoints = glob(f"../input/{exp_name}/*.pth")
    
    preds_all = []
    for checkpoint in checkpoints:
        print(f"running model {checkpoint}")
        
        model = FeedbackStackerModel(n_features=ds.num_features).to("cuda").eval()
    
        model_weights = torch.load(checkpoint, map_location="cpu")

        model.load_state_dict(collections.OrderedDict(model_weights), strict=True)
        
        del model_weights
        gc.collect() 
    
        batch_size = BS
        dl = DataLoader(ds, shuffle=False, batch_size = batch_size, num_workers = N_CORES)

        with torch.no_grad():
            preds = []
            for batch in tqdm(dl):

                data = [x.to("cuda") for x in batch]
                inputs, target = data
                out = model(inputs, target)
                preds.append(out["logits"].float().softmax(dim=1).detach().cpu().numpy())

        preds_all.append(np.concatenate(preds, axis=0))
        
        del model
        del dl
        gc.collect()
        
    del ds
    
    
    preds = np.mean(preds_all, axis=0)
    
    return preds

nn_stacker_preds_1 = run_nn_stacker("feedback-nn-v8-blend151-ff", df, BS=64)

running model ../input/feedback-nn-v8-blend151-ff/checkpoint_fold2_seed0.pth


100%|██████████| 6/6 [00:00<00:00, 21.81it/s]


running model ../input/feedback-nn-v8-blend151-ff/checkpoint_fold1_seed0.pth


100%|██████████| 6/6 [00:00<00:00, 20.25it/s]


running model ../input/feedback-nn-v8-blend151-ff/checkpoint_fold4_seed0.pth


100%|██████████| 6/6 [00:00<00:00, 21.63it/s]


running model ../input/feedback-nn-v8-blend151-ff/checkpoint_fold3_seed0.pth


100%|██████████| 6/6 [00:00<00:00, 21.74it/s]


running model ../input/feedback-nn-v8-blend151-ff/checkpoint_fold0_seed0.pth


100%|██████████| 6/6 [00:00<00:00, 21.76it/s]


In [41]:
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import torch

import torch.nn as nn

class FeedbackStackerDataset(Dataset):

    def __init__(self, df, mode):
        self.df = df.copy()
        self.mode = mode

        self.label_cols = label_cols.copy()
        
        p = [p[self.df.index.values] for p in preds_orig.copy()]
        p = np.stack(p)
        
        df = self.df
        
        X = []
        for j in range(p.shape[0]):
            cols = []
            for jj, l in enumerate(label_cols):

                df[f"oof_{l}"] = p[j,:,jj]
                cols.append(f"oof_{l}")
                
                df[f"oof_{l}_mean"] = df.groupby("essay_id")[f"oof_{l}"].transform("mean")
                cols.append(f"oof_{l}_mean")

                df[f"oof_{l}_t_mean"] = df.groupby(["essay_id", "discourse_type"])[f"oof_{l}"].transform("mean")
                cols.append(f"oof_{l}_t_mean")
                
            df[f"len"] = df.groupby("essay_id")[f"discourse_id"].transform("count") / 10
            cols.append(f"len")
        
            
            X.append(df[cols].values)
         
        X = np.stack(X).transpose(1,2,0)
        print(X.shape)
        
        self.num_features = X.shape[1]

        self.X = X
        self.y = self.df[self.label_cols].values


    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        X = self.X[idx]
        y = self.y[idx]
        
        return torch.FloatTensor(X), torch.FloatTensor(y)


    def __len__(self):
        return self.df.shape[0]

class FeedbackStackerModel(nn.Module):
    def __init__(self, n_features):
        super(FeedbackStackerModel, self).__init__()
        
        self.sizes = [256, 128, 64]
        
        layers = []
        for j,s in enumerate(self.sizes):
            if j == 0:
                layers.append(nn.Conv1d(n_features, s, 1))
            else:
                layers.append(nn.Conv1d(self.sizes[j-1], s, 1))
            layers.append(nn.PReLU())
            layers.append(nn.Dropout(0.2))
        
        self.features = nn.Sequential(*layers)
        self.head = nn.Linear(self.sizes[-1], 3)
        
        self.loss_fn = nn.CrossEntropyLoss()
        
    def forward(self, x, y):
        x = self.features(x)
        x = x.mean(dim=2)
        x = self.head(x)
        
        output = {}
        
        output["logits"] = x
        
        if self.training:
            output["loss"] = self.loss_fn(x, y.argmax(dim=1))
        
        return output
    
ds = FeedbackStackerDataset(df.copy(), mode="val")
ds[0][0].shape

nn_stacker_preds_2 = run_nn_stacker("feedback-nn-v11-blend151-ff", df, BS=64)

(358, 10, 15)
(358, 10, 15)
running model ../input/feedback-nn-v11-blend151-ff/checkpoint_fold2_seed0.pth


100%|██████████| 6/6 [00:05<00:00,  1.04it/s]


running model ../input/feedback-nn-v11-blend151-ff/checkpoint_fold1_seed0.pth


100%|██████████| 6/6 [00:00<00:00, 21.50it/s]


running model ../input/feedback-nn-v11-blend151-ff/checkpoint_fold4_seed0.pth


100%|██████████| 6/6 [00:00<00:00, 21.16it/s]


running model ../input/feedback-nn-v11-blend151-ff/checkpoint_fold3_seed0.pth


100%|██████████| 6/6 [00:00<00:00, 20.51it/s]


running model ../input/feedback-nn-v11-blend151-ff/checkpoint_fold0_seed0.pth


100%|██████████| 6/6 [00:00<00:00, 16.09it/s]


In [42]:
def gen_x(values):
    range = 1
    return np.histogram(np.clip(values, 0.001, 0.999*range), bins=3, density=True, range=(0,range))[0]

In [43]:
all_groups = []

gb = df.groupby('essay_id', sort=False)
for name, group in tqdm(gb):
    group["n_types"] = group.discourse_type.nunique()
    for class_name in ["Adequate", "Effective", "Ineffective"]:
        if class_name in ["Adequate", "Effective"]:
            continue
        for idx, val in enumerate(gen_x(group[class_name].values)):
            group[f"{class_name}_bin_{idx}"] = val 
        group[f"mean_{class_name}"] = group[class_name].mean()    

    all_groups.append(group)

df = pd.concat(all_groups).reset_index(drop=True)

disc_types_mapping = {'Lead': 0,
'Position': 1,
'Claim': 2,
'Evidence': 3,
'Counterclaim': 4,
'Rebuttal': 5,
'Concluding Statement': 6}
df["len_disc"] = df.discourse_text.str.len()

df["discourse_type"] = df["discourse_type"].map(disc_types_mapping)

df["paragraph_cnt"] = df.essay_text.map(lambda x: len(x.split("\n\n")))

100%|██████████| 41/41 [00:00<00:00, 486.90it/s]


In [44]:
import lightgbm

lgb_stacker_preds = []

for fold in range(5):
    print(fold)
    gbm = lightgbm.Booster(model_file=f"../input/lightgbm151/model_fold_{fold}.txt")
    valid_pred = gbm.predict(df[['discourse_type', 'Adequate', 'Effective', 'Ineffective', 'n_types',
       'Ineffective_bin_0', 'Ineffective_bin_1', 'Ineffective_bin_2',
       'mean_Ineffective', 'len_disc', 'paragraph_cnt']])
    lgb_stacker_preds.append(valid_pred)

0
1
2
3
4


In [45]:
lgb_stacker_preds = np.array(lgb_stacker_preds).mean(axis=0)

In [46]:
all_groups = []

gb = df_ind_models.groupby('essay_id', sort=False)
for name, group in tqdm(gb):
    group["n_types"] = group.discourse_type.nunique()

    for class_name in ["Adequate", "Effective", "Ineffective"]:
        if class_name in ["Adequate", "Effective"]:
            continue

        for idx, val in enumerate(gen_x(group[class_name].values)):
            group[f"{class_name}_bin_{idx}"] = val

        group[f"mean_{class_name}"] = group[class_name].mean()
        group[f"max_{class_name}"] = group[class_name].max()

    for class_name in [f"Ineffective_{i}" for i in range(15)]:
        group[f"mean_{class_name}"] = group[class_name].mean()

    for class_name in [f"Effective_{i}" for i in range(15)]:
        group[f"mean_{class_name}"] = group[class_name].mean()

    all_groups.append(group)

df_ind_models = pd.concat(all_groups).reset_index(drop=True)

df_ind_models["paragraph_cnt"] = df_ind_models.essay_text.map(lambda x: len(x.split("\n\n")))

disc_types_mapping = {'Lead': 0,
'Position': 1,
'Claim': 2,
'Evidence': 3,
'Counterclaim': 4,
'Rebuttal': 5,
'Concluding Statement': 6}

df_ind_models["discourse_type"] = df_ind_models["discourse_type"].map(disc_types_mapping)

for i in range(15):
    df_ind_models = df_ind_models.drop([f'Adequate_{i}'], axis=1)

100%|██████████| 41/41 [00:00<00:00, 61.30it/s]


In [47]:
import lightgbm

lgb_stacker_preds_2 = []

for fold in range(5):
    print(fold)
    gbm = lightgbm.Booster(model_file=f"../input/lightgbm151v2/model_fold_{fold}.txt")
    valid_pred = gbm.predict(df_ind_models[['discourse_type', 'Adequate', 'Effective', 'Ineffective', 'Effective_0',
       'Ineffective_0', 'Effective_1', 'Ineffective_1', 'Effective_2',
       'Ineffective_2', 'Effective_3', 'Ineffective_3', 'Effective_4',
       'Ineffective_4', 'Effective_5', 'Ineffective_5', 'Effective_6',
       'Ineffective_6', 'Effective_7', 'Ineffective_7', 'Effective_8',
       'Ineffective_8', 'Effective_9', 'Ineffective_9', 'Effective_10',
       'Ineffective_10', 'Effective_11', 'Ineffective_11', 'Effective_12',
       'Ineffective_12', 'Effective_13', 'Ineffective_13', 'Effective_14',
       'Ineffective_14', 'n_types', 'Ineffective_bin_0', 'Ineffective_bin_1',
       'Ineffective_bin_2', 'mean_Ineffective', 'max_Ineffective',
       'mean_Ineffective_0', 'mean_Ineffective_1', 'mean_Ineffective_2',
       'mean_Ineffective_3', 'mean_Ineffective_4', 'mean_Ineffective_5',
       'mean_Ineffective_6', 'mean_Ineffective_7', 'mean_Ineffective_8',
       'mean_Ineffective_9', 'mean_Ineffective_10', 'mean_Ineffective_11',
       'mean_Ineffective_12', 'mean_Ineffective_13', 'mean_Ineffective_14',
       'mean_Effective_0', 'mean_Effective_1', 'mean_Effective_2',
       'mean_Effective_3', 'mean_Effective_4', 'mean_Effective_5',
       'mean_Effective_6', 'mean_Effective_7', 'mean_Effective_8',
       'mean_Effective_9', 'mean_Effective_10', 'mean_Effective_11',
       'mean_Effective_12', 'mean_Effective_13', 'mean_Effective_14',
       'paragraph_cnt']])
    lgb_stacker_preds_2.append(valid_pred)

0
1
2
3
4


In [48]:
lgb_stacker_preds_2 = np.array(lgb_stacker_preds_2).mean(axis=0)

In [49]:
all_preds = [
    orig_preds.copy(),
    lgb_stacker_preds.copy(),
    nn_stacker_preds_1.copy(),
    nn_stacker_preds_2.copy(),
    lgb_stacker_preds_2.copy(),
]

all_preds = np.average(all_preds, axis=0, weights=[2.17532521, 0.96247677, 1.15351147, 0.62746974, 0.47835051])

In [50]:
df["Adequate"] = all_preds[:, 0]
df["Effective"] = all_preds[:, 1]
df["Ineffective"] = all_preds[:, 2]

In [51]:
df[['discourse_id', 'Ineffective', 'Adequate', 'Effective']].to_csv("submission.csv", index=False)

In [52]:
if CALC_SCORE:
    from sklearn.metrics import log_loss
    
    label_cols = ["Adequate", "Effective", "Ineffective"]
    
    y = np.zeros_like(all_preds)
    
    for ii, jj in enumerate([label_cols.index(x) for x in df["discourse_effectiveness"].values]):
        y[ii,jj] = 1
        
    print(log_loss(y, all_preds))

0.3429446601096532
